In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import the library we need

In [8]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

There are two ways to build a neural network.

In [11]:
model_1 = Sequential([
    Dense(1, input_dim=2, activation='sigmoid')])

In [12]:
model_2 = Sequential()
model_2.add(Dense(1, input_dim=2, activation='sigmoid'))

In [13]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [14]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
